# Double Variable Polynomial Regression

## Install and Import Dependencies

In [ ]:
%pip install numpy pandas matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load Data

In [ ]:
df = pd.read_csv('../DoubleVariablePolynomialRegression.ipynb/data/Fish.csv')

In [ ]:
df.head()

## Note: No NAN Values

In [ ]:
df.isna().sum()

## Normalize the Features

In [ ]:
df['Height'] = (df['Height']-np.mean(df['Height']))/np.std(df['Height'])
df['Width'] = (df['Width']-np.mean(df['Width']))/np.std(df['Width'])
df.head()

## Generate Random Test and Train Splits

In [ ]:
seed = 420
train_fraction = 0.8
train = df.sample(frac=train_fraction, random_state=seed)
test = df.drop(train.index)

## Polynomial Regression Model

In [ ]:
class PolynomialRegressionModel:
    def __init__(self, degree, q, lmbda):
        """
        Polynomial Regression Model for some particular degree.
        """
        self.train_errors = {}
        self.test_errors = {}
        self.q = q
        self.lmbda = lmbda
        self.degree = degree
        # Initialize Weights
        self.weights = np.random.rand(degree+1, degree+1)

    def calculate_loss(self, X_i, t_i):
        # print('inside self.calculate_loss()')
        assert type(X_i[0]) == np.float64 and type(t_i) == np.float64 and type(X_i[1]) == np.float64, "Types are not matching. Check!"

        a = X_i[0]
        b = X_i[1]
        t = t_i
        prediction = self.predict([(a, b)])
        # print('predicted:')
        # print(prediction)
        # print('expected')
        # print(t)

        grad = np.zeros_like(self.weights)
        grad.fill(0.0)
        for i in range(self.degree+1):
            for j in range(self.degree+1):
                if i + j <= self.degree:
                    grad[i][j] = (a**i)*(b**j)*(t - prediction)
        
        grad += (self.lmbda*self.q//2)*(np.abs(self.weights)**(self.q-1))
        # print('loss: ')
        # print(grad)
        return -1*grad

    def fit(self, X_train, y_train, X_test, y_test, lr=0.01, epochs=500, batch_size=20):
        """
        Fit the polynomial regression model using Batch Gradient Descent.

        Parameters:
        X_train: Input Feature variables.
        y_train: Target Variable
        X_test: Input Feature variables for test data
        y_test: Target Variables for test data
        lr: Learning Rate for Gradient Descent
        epochs: No of Epochs to train

        Returns:
        NA
        """
        print('Starting Training.....')
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        # print(X_train.head())
        for epoch in range(epochs):
            count = 0
            loss = np.zeros_like(self.weights)
            # print(X_train.shape[0])
            
            for i in range(X_train.shape[0]):
                # print('sample')
                # print(X_train[i][0])
                # print(X_train[i][1])
                # print(y_train[i])
                if epoch == 0 or (epoch*epochs+i)%(X_train.shape[0]/2):
                    self.train_errors[epoch*epochs + i] = self.calculate_error(X_train, y_train)
                    self.test_errors[epoch*epochs + i] = self.calculate_error(X_test, y_test)

                X_i = (X_train[i][0],X_train[i][1])
                t_i =  y_train[i]

                if count%batch_size == 0:
                    loss /= batch_size
                    # print('loss: ')
                    # print(loss)
                    # print(self.weights)
                    # self.weights += (self.lmbda*self.q//2)*(np.abs(self.weights)**(self.q-1))
                    self.weights -= lr*loss
                    # print(self.weights)
                    loss = np.zeros_like(self.weights)
                else:
                    loss += self.calculate_loss(X_i, t_i)
            
                count+=1
            if epoch%(epochs/10) == 0:
                print(f"epoch: {epoch}")
                print(f"Error: {self.calculate_error(X_train, y_train)}")

        return

    def calculate_error(self, X_test, y_test):
        """
        Find the error of the model on some data.

        Parameters:
        X_test: The sample Input Feature.
        y_test: The sample Target Feature.

        Returns:
        A float value that is the MSE b/w the predicted outputs and the target outputs.
        """
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        predictions = self.predict(X_test)
        mse = np.mean(
            (predictions-y_test)**2
        )
        return mse

    def predict(self, X_test):
        """
        Make Predictions using the trained model.

        Parameters:
        X_test: The sample Input Features.

        Returns:
        A numpy Array with the predicted target variable value for each of the samples having
        same dimensions as X_test.
        """
        result = [] 
        for sample in X_test:
            assert type(sample[0]) == np.float64 and type(sample[1]) == np.float64, "Variable doesn't have the required type!"
            degree = 2
            a = sample[0]
            b = sample[1]
            y = 0
            for i in range(degree+1):
                for j in range(degree+1):
                    if i + j <= degree:
                        y += self.weights[i][j]*(a**i)*(b**j)
            result.append(y)
        return np.array(result)

## Grid Search

## Without Regularization

In [ ]:
import json

In [ ]:
errors = []
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9]:  
    for j in  [1, 2, 3, 4, 5, 6, 7, 8, 9]:
        for lr in [0.1, 0.001, 0.0001]:
            print(f"doing: {i}, {j}, {lr} for 500 epochs")
            model = PolynomialRegressionModel(i=i, j=j, q=0, lmbda=0)
            model.fit(train.drop(['Weight'], axis=1), train['Weight'], test.drop(['Weight'], axis=1), test['Weight'], lr=lr, epochs=500)
            errors.append({
                "i": i,
                "j": j,
                "lr": lr,
                "test_errors: ": model.test_errors,
                "train_errors: ": model.train_errors
            })
            print(errors)
json_obj = json.dumps(errors)
with open('double_noreg.json', 'w') as fp:
    json.dump(json_obj, fp)

## With Regularization

In [ ]:
errors = []
# Note this should be chosen from analysis of previous grid search without regularization
best_fit_degree = 2
for degree in [best_fit_degree]:  
    for lr in [0.01, 0.001, 0.0001]:
        for q in [0.5, 1, 2, 4]:
            for batch_size in [20, 1]:
                for lmbda in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                    print(f"doing: {degree}, {lr}, {q}, {batch_size} for 500 epochs")
                    model = PolynomialRegressionModel(degree=degree, q=q, lmbda=lmbda)
                    model.fit(train.drop(['Weight'], axis=1), train['Weight'], test.drop(['Weight'], axis=1), test['Weight'], lr=lr, epochs=500, batch_size=batch_size)
                    errors.append({
                        "degree": degree,
                        "lr": lr,
                        "q": q, 
                        "lmbda": lmbda,
                        "batch_size": batch_size,
                        "test_errors: ": model.test_errors,
                        "train_errors: ": model.train_errors
                    })
                    print(errors)
json_obj = json.dumps(errors)
with open('double_withreg.json', 'w') as fp:
    json.dump(json_obj, fp)

## Plots

In [ ]:
model = PolynomialRegressionModel(degree=5, q=2, lmbda=0.1)
model.fit(train.drop(['Weight'], axis=1), train['Weight'], test.drop(['Weight'], axis=1), test['Weight'], lr=0.001, epochs=200, batch_size=2)

In [ ]:
model.test_errors
epochs = list(model.train_errors.keys())
errors = list(model.train_errors.values())

plt.plot(epochs, errors, marker='o')
epochs = list(model.test_errors.keys())
errors = list(model.test_errors.values())
plt.plot(epochs, errors, marker='x')
plt.title('Error vs Samples')
plt.xlabel('Samples')
plt.ylabel('Error')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
a_values = np.linspace(min(train.to_numpy()[:, 0]), max(train.to_numpy()[:, 0]), 100)
b_values = np.linspace(min(test.to_numpy()[:, 1]), max(train.to_numpy()[:, 1]), 100)
a_mesh, b_mesh = np.meshgrid(a_values, b_values)

prediction_points = np.c_[a_mesh.ravel(), b_mesh.ravel()]
predictions = model.predict(prediction_points)
predictions_surface = predictions.reshape(a_mesh.shape)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(a_mesh, b_mesh, predictions_surface, cmap='viridis')

# ax.scatter((train+test .to_numpy()[:, 0], train.to_numpy()[:, 1], train['Weight'].to_numpy(), color='blue', marker='o')

ax.set_xlabel('Height')
ax.set_ylabel('Width')
ax.set_zlabel('Predicted Weight')

plt.title('3D Surface Plot of Polynomial Regression Model Predictions')
plt.show()

## R2 value

In [ ]:
from __future__ import division 
import numpy as np

def compute_r2(y_true, y_predicted):
    sse = sum((y_true - y_predicted)**2)
    tse = (len(y_true) - 1) * np.var(y_true, ddof=1)
    r2_score = 1 - (sse / tse)
    return r2_score, sse, tse

y_pred = model.predict(np.array(test.drop(['Weight'], axis=1)))
y_actual = test['Weight']
compute_r2(y_actual, y_pred)[0]

In [ ]:
# Plotting the 45-degree line
plt.plot([min(y_actual), max(y_actual)], [min(y_actual), max(y_actual)], linestyle='--', color='gray', label='45-degree line')

# Scatter plot for y_pred and y_actual
plt.scatter(y_actual, y_pred, color='blue', label='Scatter plot')

# Adding labels and title
plt.xlabel('y_actual')
plt.ylabel('y_pred')
plt.title('Scatter plot of y_pred vs y_actual')

# Adding a legend
plt.legend()

# Display the plot
plt.show()